In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os 
import time
from google.cloud import bigquery
import pandas_gbq
import glob

In [2]:
client = bigquery.Client()

In [3]:
query = """SELECT * FROM `repositoriodedadosgpsp.perfil_remuneracao.SIAPE_ingresso_aposentadoria_media`"""  

In [4]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,tipo_aposentadoria,decada_aposentadoria,categoria_ministerios,media_contribuicao,quantidade_vinculos
0,Outros,1980,"Economia, Finanças, Planejamento, Previdência ...",26.0,3
1,Outros,1980,Educação e Ciência e Tecnologia,19.7,6
2,Outros,1980,Saúde,19.2,4
3,Outros,1990,"Cultura, Turismo e Esportes",12.0,1
4,Outros,1990,"Economia, Finanças, Planejamento, Previdência ...",26.6,5
...,...,...,...,...,...
200,Aposentadoria por incapacidade,2020,"Justiça, Segurança e Órgãos de Controle e Defesa",23.0,152
201,Aposentadoria por incapacidade,2020,"Meio ambiente, Agricultura e Agropecuária",27.2,37
202,Aposentadoria por incapacidade,2020,Presidência da República,20.1,51
203,Aposentadoria por incapacidade,2020,Relações Exteriores,29.9,13


In [5]:
df.columns

Index(['tipo_aposentadoria', 'decada_aposentadoria', 'categoria_ministerios',
       'media_contribuicao', 'quantidade_vinculos'],
      dtype='object')

In [11]:
df['ano'].unique()

<IntegerArray>
[2023, 2021, 2020, 2022]
Length: 4, dtype: Int64

In [12]:
df['org_lotacao'].unique()

array(['Ministério do Turismo', 'Ministério da Economia',
       'Ministério da Saúde', 'Ministério da Defesa',
       'Ministério da Educação', 'Ministério de Minas e Energia',
       'Ministério da Justiça e Segurança Pública',
       'Ministério da Infraestrutura',
       'Ministério da Ciência, Tecnologia, Inovações e Comunicações',
       'Presidência da República', 'Ministério das Relações Exteriores',
       'Ministério da Agricultura, Pecuária e Abastecimento',
       'Ministério do Desenvolvimento Regional',
       'Ministério do Meio Ambiente',
       'Ministério do Desenvolvimento Social',
       'Controladoria-Geral da União', 'MINISTERIO DAS COMUNICACOES',
       'Defensoria Pública da União',
       'Ministério da Mulher, Família e Direitos Humanos',
       'Caixa de Financiamento Imobiliário da Aeronáutica'], dtype=object)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Id_SERVIDOR_PORTAL                    1000 non-null   Int64  
 1   NOME                                  1000 non-null   object 
 2   CPF                                   1000 non-null   object 
 3   MATRICULA                             1000 non-null   object 
 4   COD_TIPO_APOSENTADORIA                1000 non-null   Int64  
 5   TIPO_APOSENTADORIA                    1000 non-null   object 
 6   DATA_APOSENTADORIA                    1000 non-null   object 
 7   DESCRICAO_CARGO                       1000 non-null   object 
 8   COD_UORG_LOTACAO                      1000 non-null   Int64  
 9   UORG_LOTACAO                          1000 non-null   object 
 10  COD_ORG_LOTACAO                       1000 non-null   Int64  
 11  ORG_LOTACAO       

In [8]:
dir = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar'

In [9]:
os.chdir(dir)

In [13]:
df.to_csv('SIAPE_ingresso_aposentadoria_media.csv', sep=';', encoding='latin-1', index=False)

-----------------------------------------------------------

Testes

In [14]:
df = pd.read_csv("SIAPE_ingresso_aposentadoria_media.csv", encoding="latin-1", sep=';', decimal=',')
df

,tipo_aposentadoria,decada_aposentadoria,categoria_ministerios,media_contribuicao,quantidade_vinculos
0,Outros,1980,"Economia, Finanças, Planejamento, Previdência ...",26.0,3
1,Outros,1980,Educação e Ciência e Tecnologia,19.7,6
2,Outros,1980,Saúde,19.2,4
3,Outros,1990,"Cultura, Turismo e Esportes",12.0,1
4,Outros,1990,"Economia, Finanças, Planejamento, Previdência ...",26.6,5
...,...,...,...,...,...
200,Aposentadoria por incapacidade,2020,"Justiça, Segurança e Órgãos de Controle e Defesa",23.0,152
201,Aposentadoria por incapacidade,2020,"Meio ambiente, Agricultura e Agropecuária",27.2,37
202,Aposentadoria por incapacidade,2020,Presidência da República,20.1,51
203,Aposentadoria por incapacidade,2020,Relações Exteriores,29.9,13


Automatizando troca de csv

In [45]:
##os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [46]:
## baixando os atuais com , e exportando com ";"

In [47]:
## listando todos os itens do diretório
##len(glob.glob('*.csv'))

In [48]:
## Serve apenas na primeira vez . 

##for i in list(range(0,len(glob.glob('*.csv')))):
    
   # df= pd.read_csv(glob.glob('*.csv')[i])
    #df.to_csv(glob.glob('*.csv')[i],index=False, encoding="latin-1", sep=';', decimal=',')